In [ ]:
from google.colab import drive
import os

# Attempt to unmount first to clear any issues, then mount
drive.flush_and_unmount()

# Forcefully remove the mount point directory if it exists and is not empty
if os.path.exists('/content/drive') and os.path.isdir('/content/drive') and os.listdir('/content/drive'):
    !rm -rf /content/drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Cloning GIT to call dependencies and envrionments

In [ ]:
!rm -rf Assignment-3
!git clone https://github.com/murtazadroid/Assignment-3.git
%cd /content/Assignment-3
!git log -1 --oneline

Cloning into 'Assignment-3'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 73 (delta 32), reused 62 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (73/73), 290.86 KiB | 24.24 MiB/s, done.
Resolving deltas: 100% (32/32), done.
/content/Assignment-3
1cf9ab0 (HEAD -> main, origin/main, origin/HEAD) Commit 7: 1) Environment Design changed by, changing the action space. We removed the action of Selling a YES contract and replaced it with adding a action to buy NO contract if better rewards.


In [ ]:
%cd /content/Assignment-3
!ls

/content/Assignment-3
agent  backtest  env  models  README.md  requirements.txt  tests


# Install Requirement from requirements.txt


In [ ]:
!pip -q install -r requirements.txt


Reason for being yanked: Loading broken with PyTorch 1.13
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 18.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.1 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd

from env.data_loader import generate_synthetic_btc_minute_data, split_into_days
from env.kalshi_env import KalshiBTCHourlyEnv
from backtest.baselines import NoTradePolicy, RandomPolicy, ProbHeuristicPolicy

print("✅ Imports OK")


✅ Imports OK


# Check if all paths okay

In [ ]:
import sys
from pathlib import Path

ROOT = Path("/content/Assignment-3")
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("PYTHONPATH OK:", sys.path[0])


PYTHONPATH OK: /content/Assignment-3


# Set the environment configuration

In [ ]:
from env.kalshi_env import KalshiBTCHourlyEnv
from env.data_loader import generate_synthetic_btc_minute_data, split_into_days
from env.config import ENV_KWARGS

print("ENV_KWARGS:", ENV_KWARGS)
print("Imports OK ✅")


ENV_KWARGS: {'start_balance': 10000.0, 'fee_per_contract': 0.001, 'spread': 0.02, 'strike_step': 100.0, 'decision_offset_minutes': 60, 'trade_penalty_coef': 0.001, 'pricing_k': 5.0, 'pricing_bias': 0.0, 'pricing_noise_std': 0.06}
Imports OK ✅


# Environment first intializtion and check

In [ ]:
df = generate_synthetic_btc_minute_data(num_days=30, seed=123)
day_list = split_into_days(df)

env = KalshiBTCHourlyEnv(day_data_list=day_list, **ENV_KWARGS)

obs, info = env.reset(seed=123)
print("Obs shape:", obs.shape)
print("Sample obs:", obs[:5])

total_r = 0.0
for t in range(15):
    action = env.action_space.sample()
    obs, r, term, trunc, info = env.step(action)
    total_r += r
    if t < 5:
        print(f"t={t} action={action} r={r:.4f} pv={info.get('portfolio_value'):.2f}")
    if term or trunc:
        break

print("Episode total reward:", total_r)
print("Final info:", info)


Obs shape: (11,)
Sample obs: [-4.5159642e-04 -1.3100944e-03 -9.8337012e-04  5.0976436e-04
  1.0000000e+00]
t=0 action=1 r=-0.5887 pv=9999.41
t=1 action=1 r=0.4337 pv=9999.85
t=2 action=1 r=0.5255 pv=10000.37
t=3 action=1 r=-0.4936 pv=9999.88
t=4 action=0 r=0.0000 pv=9999.88
Episode total reward: 1.6257792056719045
Final info: {'hour': 23, 'decision_price': 50595.29654371957, 'threshold': 50600.0, 'resolve_price': 50569.32916485338, 'payoff_yes': 0.0, 'payoff_no': 1.0, 'yes_midprice': 0.4781777955706468, 'yes_bid': 0.4681777955706468, 'yes_ask': 0.48817779557064683, 'no_midprice': 0.5218222044293532, 'no_bid': 0.5118222044293532, 'no_ask': 0.5318222044293532, 'trade_side': -1, 'trade_cost': 0.5328222044293532, 'portfolio_value': 10001.637779205672}


/content/Assignment-3/env/data_loader.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  timestamps = pd.date_range(


# Trying differnent policies for Environment Sanity check


In [ ]:
import numpy as np
from backtest.baselines import NoTradePolicy, RandomPolicy, ProbHeuristicPolicy

def evaluate_policy(env, policy, num_episodes=50):
    pnls = []
    start_balance = env.start_balance
    for _ in range(num_episodes):
        obs, _ = env.reset()
        final_pv = start_balance
        while True:
            a = policy.select_action(obs)
            obs, r, term, trunc, info = env.step(a)
            final_pv = info["portfolio_value"]
            if term or trunc:
                pnls.append(final_pv - start_balance)
                break
    pnls = np.array(pnls)
    return {
        "mean_pnl": float(pnls.mean()),
        "std_pnl": float(pnls.std()),
        "win_rate": float((pnls > 0).mean()),
        "min_pnl": float(pnls.min()),
        "max_pnl": float(pnls.max()),
    }

no_trade = NoTradePolicy()
rand = RandomPolicy(seed=0)
heur = ProbHeuristicPolicy()

print("NoTrade:", evaluate_policy(env, no_trade))
print("Random :", evaluate_policy(env, rand))
print("Heur.  :", evaluate_policy(env, heur))


NoTrade: {'mean_pnl': 0.0, 'std_pnl': 0.0, 'win_rate': 0.0, 'min_pnl': 0.0, 'max_pnl': 0.0}
Random : {'mean_pnl': -0.1468934158268894, 'std_pnl': 1.4582482552963396, 'win_rate': 0.5, 'min_pnl': -4.081241539677649, 'max_pnl': 3.6110669197278185}
Heur.  : {'mean_pnl': -0.05680706247690978, 'std_pnl': 1.1075262460855555, 'win_rate': 0.4, 'min_pnl': -2.1762786261897418, 'max_pnl': 2.4531021849234094}


#PPO training env (VecNormalize + Monitor)

In [ ]:
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

def make_train_env(seed=123, num_days=200):
    df = generate_synthetic_btc_minute_data(num_days=num_days, seed=seed)
    day_list = split_into_days(df)

    def _init():
        e = KalshiBTCHourlyEnv(day_data_list=day_list, **ENV_KWARGS)
        e.reset(seed=seed)
        return Monitor(e)

    venv = DummyVecEnv([_init])
    venv = VecNormalize(venv, norm_obs=True, norm_reward=True, clip_obs=10.0, gamma=0.995)
    return venv

train_env = make_train_env(seed=123, num_days=200)
print("Train env ready ✅")


Train env ready ✅


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/content/Assignment-3/env/data_loader.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  timestamps = pd.date_range(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


#Declaring Directory for model and configuration saving


In [ ]:
import os, time, json, subprocess
from pathlib import Path

def now():
    return time.strftime("%Y-%m-%d_%H-%M-%S")

BASE_DIR = "/content/drive/MyDrive/kalshi_rl"
RUN_TYPE = "sanity"   # "sanity" or "train"
RUN_NAME = "ppo"      # e.g., "ppo", "ppo_long", "ppo_ablation"

# Organized run folder
STAMP = now()
RUN_ID = f"{RUN_TYPE}__{RUN_NAME}__{STAMP}"
RUN_DIR = Path(BASE_DIR) / "runs" / RUN_ID

DIRS = {
    "tb": RUN_DIR / "tb",
    "logs": RUN_DIR / "logs",
    "checkpoints": RUN_DIR / "checkpoints",
    "final": RUN_DIR / "final",
    "meta": RUN_DIR / "meta",
}
for d in DIRS.values():
    d.mkdir(parents=True, exist_ok=True)

# Convenience pointer to last run of this type/name
LATEST_DIR = Path(BASE_DIR) / "runs" / "latest" / f"{RUN_TYPE}__{RUN_NAME}"
LATEST_DIR.parent.mkdir(parents=True, exist_ok=True)

# Make a "latest" marker file (works everywhere; symlinks can be flaky on Drive)
LATEST_TXT = LATEST_DIR.with_suffix(".txt")
LATEST_TXT.write_text(str(RUN_DIR))
print("Run directory:", RUN_DIR)
print("Latest pointer:", LATEST_TXT)

# --- Save reproducibility metadata ---
def try_cmd(cmd):
    try:
        return subprocess.check_output(cmd, stderr=subprocess.STDOUT).decode().strip()
    except Exception as e:
        return f"ERROR: {e}"

meta = {
    "run_id": RUN_ID,
    "run_dir": str(RUN_DIR),
    "timestamp": STAMP,
    "run_type": RUN_TYPE,
    "run_name": RUN_NAME,
    "git_commit": try_cmd(["git", "rev-parse", "HEAD"]),
    "git_status": try_cmd(["git", "status", "--porcelain"]),
    "pip_freeze": try_cmd(["pip", "freeze"]),
}

# Save metadata
(DIRS["meta"] / "run_meta.json").write_text(json.dumps(meta, indent=2))

# If you have env config dict available, save it too:
# from env.config import ENV_KWARGS
# (DIRS["meta"] / "env_config.json").write_text(json.dumps(ENV_KWARGS, indent=2))

print("Saved meta to:", DIRS["meta"] / "run_meta.json")


Run directory: /content/drive/MyDrive/kalshi_rl/runs/sanity__ppo__2025-12-20_01-06-13
Latest pointer: /content/drive/MyDrive/kalshi_rl/runs/latest/sanity__ppo.txt


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Saved meta to: /content/drive/MyDrive/kalshi_rl/runs/sanity__ppo__2025-12-20_01-06-13/meta/run_meta.json


#Training PP0 for few runs for training sanity check

In [ ]:
from pathlib import Path
import numpy as np

from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

from env.data_loader import generate_synthetic_btc_minute_data, split_into_days
from env.kalshi_env import KalshiBTCHourlyEnv
from env.config import ENV_KWARGS

SANITY_SEED = 123
SANITY_DAYS = 200
SANITY_STEPS = 100_000   # 100k is enough to see learning

# Build train data
df = generate_synthetic_btc_minute_data(num_days=SANITY_DAYS, seed=SANITY_SEED)
day_list = split_into_days(df)

def _init():
    e = KalshiBTCHourlyEnv(day_data_list=day_list, **ENV_KWARGS)
    e.reset(seed=SANITY_SEED)
    return Monitor(e)

train_env = DummyVecEnv([_init])
train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10.0, gamma=0.995)

# Train PPO
model = PPO(
    "MlpPolicy",
    train_env,
    learning_rate=3e-4,
    gamma=0.995,
    n_steps=2048,
    batch_size=512,
    n_epochs=10,
    gae_lambda=0.98,
    clip_range=0.2,
    ent_coef=0.05,
    vf_coef=0.5,
    policy_kwargs=dict(net_arch=dict(pi=[128, 128], vf=[128, 128])),
    verbose=1,
    seed=SANITY_SEED,
    device="auto",
)

print("Training sanity PPO...")
model.learn(total_timesteps=SANITY_STEPS)

# Save locally (or change to Drive RUN_DIR if you want)
Path("models").mkdir(exist_ok=True)
model.save("models/ppo_sanity")
train_env.save("models/ppo_sanity_vecnorm.pkl")

print("✅ Saved models/ppo_sanity.zip and models/ppo_sanity_vecnorm.pkl")


/content/Assignment-3/env/data_loader.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  timestamps = pd.date_range(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Using cuda device
Training sanity PPO...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 15       |
|    ep_rew_mean     | -0.358   |
| time/              |          |
|    fps             | 283      |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 15          |
|    ep_rew_mean          | 0.0854      |
| time/                   |             |
|    fps                  | 283         |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.005636231 |
|    clip_fraction        | 0.00947     |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.1        |
|    explained_variance   | -0.04725194 |
|    learning_rate        | 0.

# Sanity Evaluation

In [ ]:
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from backtest.baselines import NoTradePolicy, RandomPolicy, ProbHeuristicPolicy

EVAL_SEED = 999
EVAL_DAYS = 200
EVAL_EPISODES = 100

# Fixed eval dataset
df_eval = generate_synthetic_btc_minute_data(num_days=EVAL_DAYS, seed=EVAL_SEED)
day_list_eval = split_into_days(df_eval)

def make_eval_env_raw():
    return KalshiBTCHourlyEnv(day_data_list=day_list_eval, **ENV_KWARGS)

eval_env_raw = make_eval_env_raw()

# Load vecnorm for obs normalization
def _init_eval():
    return Monitor(make_eval_env_raw())

venv_eval = DummyVecEnv([_init_eval])
venv_eval = VecNormalize.load("models/ppo_sanity_vecnorm.pkl", venv_eval)
venv_eval.training = False
venv_eval.norm_reward = False

ppo_loaded = PPO.load("models/ppo_sanity.zip")

class PPOPolicy:
    def select_action(self, obs):
        obs_norm = venv_eval.normalize_obs(obs[None, :])
        a, _ = ppo_loaded.predict(obs_norm, deterministic=True)
        return int(np.asarray(a).item())

def evaluate_policy(env, policy, episodes=100):
    pnls = []
    start = env.start_balance
    for _ in range(episodes):
        obs, _ = env.reset()
        pv = start
        while True:
            a = policy.select_action(obs)
            obs, _, term, trunc, info = env.step(a)
            pv = info["portfolio_value"]
            if term or trunc:
                pnls.append(pv - start)
                break
    pnls = np.array(pnls)
    return {
        "mean_pnl": float(pnls.mean()),
        "std_pnl": float(pnls.std()),
        "win_rate": float((pnls > 0).mean()),
        "min_pnl": float(pnls.min()),
        "max_pnl": float(pnls.max()),
    }

# Baselines
print("NoTrade:", evaluate_policy(eval_env_raw, NoTradePolicy(), EVAL_EPISODES))
print("Random :", evaluate_policy(eval_env_raw, RandomPolicy(seed=0), EVAL_EPISODES))
print("Heur.  :", evaluate_policy(eval_env_raw, ProbHeuristicPolicy(), EVAL_EPISODES))

# PPO sanity
ppo_policy = PPOPolicy()
print("PPO_sanity:", evaluate_policy(eval_env_raw, ppo_policy, EVAL_EPISODES))

# Quick action sample (should not be all 0)
obs, _ = eval_env_raw.reset()
acts = []
for _ in range(15):
    a = ppo_policy.select_action(obs)
    acts.append(a)
    obs, _, term, trunc, _ = eval_env_raw.step(a)
    if term or trunc:
        break
print("Sample PPO actions:", acts)


/content/Assignment-3/env/data_loader.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  timestamps = pd.date_range(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


NoTrade: {'mean_pnl': 0.0, 'std_pnl': 0.0, 'win_rate': 0.0, 'min_pnl': 0.0, 'max_pnl': 0.0}
Random : {'mean_pnl': -0.12907347514315914, 'std_pnl': 1.6315619719715608, 'win_rate': 0.42, 'min_pnl': -3.411730029147293, 'max_pnl': 3.653035705210641}
Heur.  : {'mean_pnl': -0.1333534264461923, 'std_pnl': 1.144376715676714, 'win_rate': 0.45, 'min_pnl': -3.0408914325089427, 'max_pnl': 2.498128251541857}
PPO_sanity: {'mean_pnl': 0.8015759794929544, 'std_pnl': 1.793139566705328, 'win_rate': 0.66, 'min_pnl': -3.64883241780808, 'max_pnl': 4.894397136205953}
Sample PPO actions: [2, 1, 1, 0, 1, 2, 2, 1, 1, 2, 1, 1, 0, 0, 1]


# Main Agent Training


In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback

TOTAL_TIMESTEPS = 3_000_000       # set 2M–5M
CHECKPOINT_EVERY = 200_000

checkpoint_cb = CheckpointCallback(
    save_freq=CHECKPOINT_EVERY,
    save_path=str(RUN_DIR / "checkpoints"),
    name_prefix="ppo_ckpt"
)

model = PPO(
    "MlpPolicy",
    train_env,
    learning_rate=3e-4,
    gamma=0.995,
    n_steps=2048,
    batch_size=512,
    n_epochs=10,
    gae_lambda=0.98,
    clip_range=0.2,
    ent_coef=0.05,
    vf_coef=0.5,
    policy_kwargs=dict(net_arch=dict(pi=[128,128], vf=[128,128])),
    verbose=1,
    seed=123,
    device="auto",
    tensorboard_log=str(RUN_DIR / "tb"),
)

model.learn(total_timesteps=TOTAL_TIMESTEPS, callback=checkpoint_cb)

# Save final
model.save(str(RUN_DIR / "final" / "ppo_final"))
train_env.save(str(RUN_DIR / "final" / "ppo_final_vecnorm.pkl"))

print("✅ Saved final model + vecnorm in:", RUN_DIR / "final")


Streaming output truncated to the last 5000 lines.
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 15           |
|    ep_rew_mean          | 3.56         |
| time/                   |              |
|    fps                  | 316          |
|    iterations           | 1228         |
|    time_elapsed         | 7934         |
|    total_timesteps      | 2514944      |
| train/                  |              |
|    approx_kl            | 0.0076108803 |
|    clip_fraction        | 0.0549       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.33        |
|    explained_variance   | 0.48012525   |
|    learning_rate        | 0.0003       |
|    loss                 | 0.203        |
|    n_updates            | 12270        |
|    policy_gradient_loss | -0.018       |
|    value_loss           | 0.553        |
------------------------------------------
----

# Sanity check for all saved training logs

In [ ]:
from pathlib import Path

print("RUN_DIR:", RUN_DIR)
print("\nFinal:")
for p in sorted((RUN_DIR/"final").glob("*")):
    print(" ", p.name, p.stat().st_size)

print("\nCheckpoints (first 15):")
ckpts = sorted((RUN_DIR/"checkpoints").glob("*.zip"))
for p in ckpts[:15]:
    print(" ", p.name, p.stat().st_size)

print("\n#checkpoints:", len(ckpts))


RUN_DIR: /content/drive/MyDrive/kalshi_rl/runs/sanity__ppo__2025-12-20_01-06-13

Final:
  ppo_final.zip 473243
  ppo_final_vecnorm.pkl 2039

Checkpoints (first 15):
  ppo_ckpt_1000000_steps.zip 473237
  ppo_ckpt_1200000_steps.zip 473237
  ppo_ckpt_1400000_steps.zip 473237
  ppo_ckpt_1600000_steps.zip 473237
  ppo_ckpt_1800000_steps.zip 473237
  ppo_ckpt_2000000_steps.zip 473237
  ppo_ckpt_200000_steps.zip 473235
  ppo_ckpt_2200000_steps.zip 473239
  ppo_ckpt_2400000_steps.zip 473238
  ppo_ckpt_2600000_steps.zip 473239
  ppo_ckpt_2800000_steps.zip 473239
  ppo_ckpt_3000000_steps.zip 473241
  ppo_ckpt_400000_steps.zip 473225
  ppo_ckpt_600000_steps.zip 473236
  ppo_ckpt_800000_steps.zip 473225

#checkpoints: 15


# Evaluate on baselines and FINAL model

In [ ]:
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

from env.data_loader import generate_synthetic_btc_minute_data, split_into_days
from env.kalshi_env import KalshiBTCHourlyEnv
from env.config import ENV_KWARGS
from backtest.baselines import NoTradePolicy, RandomPolicy, ProbHeuristicPolicy

EVAL_SEED = 999
EVAL_DAYS = 400
EVAL_EPISODES = 200

df_eval = generate_synthetic_btc_minute_data(num_days=EVAL_DAYS, seed=EVAL_SEED)
day_list_eval = split_into_days(df_eval)

def make_eval_env_raw():
    return KalshiBTCHourlyEnv(day_data_list=day_list_eval, **ENV_KWARGS)

eval_env_raw = make_eval_env_raw()

def _init_eval():
    return Monitor(make_eval_env_raw())

# load vecnorm from your final run
vecnorm_path = RUN_DIR / "final" / "ppo_final_vecnorm.pkl"
venv_eval = DummyVecEnv([_init_eval])
venv_eval = VecNormalize.load(str(vecnorm_path), venv_eval)
venv_eval.training = False
venv_eval.norm_reward = False

model_path = RUN_DIR / "final" / "ppo_final.zip"
ppo = PPO.load(str(model_path))

class PPOPolicy:
    def select_action(self, obs):
        obs_norm = venv_eval.normalize_obs(obs[None, :])
        a, _ = ppo.predict(obs_norm, deterministic=True)
        return int(np.asarray(a).item())

def evaluate_policy(env, policy, episodes=200):
    pnls = []
    start = env.start_balance
    for _ in range(episodes):
        obs, _ = env.reset()
        pv = start
        while True:
            a = policy.select_action(obs)
            obs, _, term, trunc, info = env.step(a)
            pv = info["portfolio_value"]
            if term or trunc:
                pnls.append(pv - start)
                break
    pnls = np.array(pnls)
    return {
        "mean_pnl": float(pnls.mean()),
        "std_pnl": float(pnls.std()),
        "win_rate": float((pnls > 0).mean()),
        "min_pnl": float(pnls.min()),
        "max_pnl": float(pnls.max()),
    }

results = {
    "NoTrade": evaluate_policy(eval_env_raw, NoTradePolicy(), EVAL_EPISODES),
    "Random":  evaluate_policy(eval_env_raw, RandomPolicy(seed=0), EVAL_EPISODES),
    "Heur":    evaluate_policy(eval_env_raw, ProbHeuristicPolicy(), EVAL_EPISODES),
    "PPO_final": evaluate_policy(eval_env_raw, PPOPolicy(), EVAL_EPISODES),
}
results


/content/Assignment-3/env/data_loader.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  timestamps = pd.date_range(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


{'NoTrade': {'mean_pnl': 0.0,
  'std_pnl': 0.0,
  'win_rate': 0.0,
  'min_pnl': 0.0,
  'max_pnl': 0.0},
 'Random': {'mean_pnl': -0.08724727974960843,
  'std_pnl': 1.707055430474854,
  'win_rate': 0.45,
  'min_pnl': -4.793694275669623,
  'max_pnl': 4.669218849257959},
 'Heur': {'mean_pnl': -0.006695575442890913,
  'std_pnl': 1.2660586966746066,
  'win_rate': 0.485,
  'min_pnl': -3.6434870366865653,
  'max_pnl': 2.863610309035721},
 'PPO_final': {'mean_pnl': -0.10750450437923063,
  'std_pnl': 1.9269251129845133,
  'win_rate': 0.465,
  'min_pnl': -4.568043048893742,
  'max_pnl': 4.700803791098224}}

# Searching for best model

In [ ]:
import numpy as np
from stable_baselines3 import PPO

ckpts = sorted((RUN_DIR/"checkpoints").glob("*.zip"))
assert len(ckpts) > 0, "No checkpoints found."

def eval_model_zip(zip_path, episodes=120):
    ppo_tmp = PPO.load(str(zip_path))
    class Pol:
        def select_action(self, obs):
            obs_norm = venv_eval.normalize_obs(obs[None, :])
            a, _ = ppo_tmp.predict(obs_norm, deterministic=True)
            return int(np.asarray(a).item())
    return evaluate_policy(eval_env_raw, Pol(), episodes)

scores = []
for i, z in enumerate(ckpts):
    r = eval_model_zip(z, episodes=120)
    scores.append((z, r["mean_pnl"], r["win_rate"], r["std_pnl"]))
    if (i+1) % 5 == 0:
        print(f"Evaluated {i+1}/{len(ckpts)}")

best = max(scores, key=lambda x: x[1])  # by mean_pnl
best


Evaluated 5/15
Evaluated 10/15
Evaluated 15/15


(PosixPath('/content/drive/MyDrive/kalshi_rl/runs/sanity__ppo__2025-12-20_01-06-13/checkpoints/ppo_ckpt_600000_steps.zip'),
 0.5454640835580904,
 0.6666666666666666,
 1.7260034264982476)

In [ ]:
from pathlib import Path

print("MODEL:", RUN_DIR/"final"/"ppo_final.zip", (RUN_DIR/"final"/"ppo_final.zip").exists())
print("VECN :", RUN_DIR/"final"/"ppo_final_vecnorm.pkl", (RUN_DIR/"final"/"ppo_final_vecnorm.pkl").exists())

print("sizes:")
print(" model bytes:", (RUN_DIR/"final"/"ppo_final.zip").stat().st_size)
print(" vecn  bytes:", (RUN_DIR/"final"/"ppo_final_vecnorm.pkl").stat().st_size)


MODEL: /content/drive/MyDrive/kalshi_rl/runs/sanity__ppo__2025-12-20_01-06-13/final/ppo_final.zip True
VECN : /content/drive/MyDrive/kalshi_rl/runs/sanity__ppo__2025-12-20_01-06-13/final/ppo_final_vecnorm.pkl True
sizes:
 model bytes: 473243
 vecn  bytes: 2039


In [ ]:
import numpy as np
from stable_baselines3 import PPO

ppo = PPO.load(str(RUN_DIR/"final"/"ppo_final.zip"))

obs, _ = eval_env_raw.reset()
acts = []
for _ in range(15):
    obs_norm = venv_eval.normalize_obs(obs[None, :])
    a, _ = ppo.predict(obs_norm, deterministic=True)
    a = int(np.asarray(a).item())
    acts.append(a)
    obs, _, term, trunc, _ = eval_env_raw.step(a)
    if term or trunc:
        break

print("Sample actions:", acts)
print("Action counts:", {i: acts.count(i) for i in set(acts)})


Sample actions: [1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2]
Action counts: {1: 8, 2: 7}


# Production Agent

In [ ]:
import shutil
from pathlib import Path

RUN_DIR = Path("/content/drive/MyDrive/kalshi_rl/runs/sanity__ppo__2025-12-20_01-06-13")
best_zip = RUN_DIR/"checkpoints"/"ppo_ckpt_600000_steps.zip"
vecnorm = RUN_DIR/"final"/"ppo_final_vecnorm.pkl"

PROD_DIR = RUN_DIR/"production"
PROD_DIR.mkdir(parents=True, exist_ok=True)

shutil.copy2(best_zip, PROD_DIR/"ppo_best.zip")
shutil.copy2(vecnorm, PROD_DIR/"ppo_best_vecnorm.pkl")

print("✅ Saved production model to:", PROD_DIR)
print("model:", PROD_DIR/"ppo_best.zip")
print("vecn :", PROD_DIR/"ppo_best_vecnorm.pkl")


✅ Saved production model to: /content/drive/MyDrive/kalshi_rl/runs/sanity__ppo__2025-12-20_01-06-13/production
model: /content/drive/MyDrive/kalshi_rl/runs/sanity__ppo__2025-12-20_01-06-13/production/ppo_best.zip
vecn : /content/drive/MyDrive/kalshi_rl/runs/sanity__ppo__2025-12-20_01-06-13/production/ppo_best_vecnorm.pkl


# Confirming the production model by reevaluating

In [ ]:
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor

from env.data_loader import generate_synthetic_btc_minute_data, split_into_days
from env.kalshi_env import KalshiBTCHourlyEnv
from env.config import ENV_KWARGS

EVAL_SEED = 999
EVAL_DAYS = 400
EVAL_EPISODES = 300

df_eval = generate_synthetic_btc_minute_data(num_days=EVAL_DAYS, seed=EVAL_SEED)
day_list_eval = split_into_days(df_eval)

def make_env():
    return KalshiBTCHourlyEnv(day_data_list=day_list_eval, **ENV_KWARGS)

eval_env_raw = make_env()

venv = DummyVecEnv([lambda: Monitor(make_env())])
venv = VecNormalize.load(str(PROD_DIR/"ppo_best_vecnorm.pkl"), venv)
venv.training = False
venv.norm_reward = False

ppo = PPO.load(str(PROD_DIR/"ppo_best.zip"))

class Pol:
    def select_action(self, obs):
        obs_norm = venv.normalize_obs(obs[None, :])
        a, _ = ppo.predict(obs_norm, deterministic=True)
        return int(np.asarray(a).item())

def eval_pnls(env, policy, episodes=300):
    pnls = []
    start = env.start_balance
    for _ in range(episodes):
        obs, _ = env.reset()
        pv = start
        while True:
            a = policy.select_action(obs)
            obs, _, term, trunc, info = env.step(a)
            pv = info["portfolio_value"]
            if term or trunc:
                pnls.append(pv - start)
                break
    pnls = np.array(pnls)
    return float(pnls.mean()), float(pnls.std()), float((pnls>0).mean())

mean_pnl, std_pnl, win = eval_pnls(eval_env_raw, Pol(), EVAL_EPISODES)
print("PPO_BEST:", {"mean_pnl": mean_pnl, "std_pnl": std_pnl, "win_rate": win})


/content/Assignment-3/env/data_loader.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  timestamps = pd.date_range(


PPO_BEST: {'mean_pnl': 0.24587274984810695, 'std_pnl': 1.863387892078347, 'win_rate': 0.57}


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# Agent Robustness check

In [ ]:
import json
import numpy as np
from pathlib import Path

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor

from env.data_loader import generate_synthetic_btc_minute_data, split_into_days
from env.kalshi_env import KalshiBTCHourlyEnv
from env.config import ENV_KWARGS

# Paths (adjust if needed)
RUN_DIR = Path("/content/drive/MyDrive/kalshi_rl/runs/sanity__ppo__2025-12-20_01-06-13")
PROD_DIR = RUN_DIR / "production"
MODEL_ZIP = PROD_DIR / "ppo_best.zip"
VECN_PKL  = PROD_DIR / "ppo_best_vecnorm.pkl"

assert MODEL_ZIP.exists(), f"Missing {MODEL_ZIP}"
assert VECN_PKL.exists(), f"Missing {VECN_PKL}"

ppo = PPO.load(str(MODEL_ZIP))

def evaluate_policy(env, select_action_fn, episodes=200):
    pnls = []
    start = env.start_balance
    for _ in range(episodes):
        obs, _ = env.reset()
        pv = start
        while True:
            a = select_action_fn(obs)
            obs, _, term, trunc, info = env.step(a)
            pv = info["portfolio_value"]
            if term or trunc:
                pnls.append(pv - start)
                break
    pnls = np.array(pnls, dtype=float)
    return {
        "mean_pnl": float(pnls.mean()),
        "std_pnl": float(pnls.std()),
        "win_rate": float((pnls > 0).mean()),
        "min_pnl": float(pnls.min()),
        "max_pnl": float(pnls.max()),
    }

def eval_seed(seed, days=400, episodes=300):
    df = generate_synthetic_btc_minute_data(num_days=days, seed=seed)
    day_list = split_into_days(df)

    def make_raw():
        return KalshiBTCHourlyEnv(day_data_list=day_list, **ENV_KWARGS)

    env_raw = make_raw()

    venv = DummyVecEnv([lambda: Monitor(make_raw())])
    venv = VecNormalize.load(str(VECN_PKL), venv)
    venv.training = False
    venv.norm_reward = False

    def select_action(obs):
        obs_norm = venv.normalize_obs(obs[None, :])
        a, _ = ppo.predict(obs_norm, deterministic=True)
        return int(np.asarray(a).item())

    return evaluate_policy(env_raw, select_action, episodes=episodes)

SEEDS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
DAYS_PER_SEED = 400
EPISODES_PER_SEED = 300

seed_results = {}
for i, s in enumerate(SEEDS, 1):
    seed_results[str(s)] = eval_seed(s, days=DAYS_PER_SEED, episodes=EPISODES_PER_SEED)
    print(f"done {i}/{len(SEEDS)} seed={s} -> mean_pnl={seed_results[str(s)]['mean_pnl']:.4f}, win={seed_results[str(s)]['win_rate']:.2%}")

# Save raw results
(PROD_DIR).mkdir(parents=True, exist_ok=True)
out_path = PROD_DIR / "robustness_eval.json"
out_path.write_text(json.dumps(seed_results, indent=2))
print("✅ Saved:", out_path)


/content/Assignment-3/env/data_loader.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  timestamps = pd.date_range(


done 1/10 seed=0 -> mean_pnl=0.4184, win=61.33%


/content/Assignment-3/env/data_loader.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  timestamps = pd.date_range(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


done 2/10 seed=1 -> mean_pnl=0.8791, win=67.67%


/content/Assignment-3/env/data_loader.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  timestamps = pd.date_range(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


...

/content/Assignment-3/env/data_loader.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  timestamps = pd.date_range(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


done 9/10 seed=8 -> mean_pnl=0.2962, win=59.00%


/content/Assignment-3/env/data_loader.py:23: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  timestamps = pd.date_range(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


done 10/10 seed=9 -> mean_pnl=0.0856, win=54.00%
✅ Saved: /content/drive/MyDrive/kalshi_rl/runs/sanity__ppo__2025-12-20_01-06-13/production/robustness_eval.json


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
